## Chapter 16: Advanced Function Features of Julia

This chapter covers a bit more on functions in julia. These feature allow the ability to write code that easier to use, read and debug. We will perform error checking for arguments to ensure that only valid arguments are considered. Additionally, to make functions more robust, we’ll use option arguments and keyword arguments.

#### 16.1: Testing Arguments

Recall the recursive factorial function:

In [3]:
fact(n::Integer) = n==0 ? 1 : n*fact(n-1)

fact (generic function with 1 method)

If we put in a negative number, then

In [4]:
fact(-5)

LoadError: StackOverflowError:

This occurs because fact(-5) would return -5 times fact(-6) which would return -5 times -6 times fact(-7), etc.  This will never stop. 

To prevent this, we will check if the argument n is greater than or equal to 0. The first line of the function, evaluates `n>=0`. If this is true, it skips the rest of the line.  If not, it throws an error. 

In [5]:
function fact(n::Integer)
  n>=0 || throw(ArgumentError("The argument must be a nonnegative integer."))
  n==0 ? 1 : n*fact(n-1)
end

fact (generic function with 1 method)

In [6]:
fact(-5)

LoadError: ArgumentError: The argument must be a nonnegative integer.

#### 16.2: Optional arguments

Recall Newton's method that we saw earlier:

In [10]:
using ForwardDiff
function newton(f::Function, x0::Number)
  local dx = f(x0)/ForwardDiff.derivative(f,x0) 
  local steps = 0
  while abs(dx)>1e-6 && steps<10
    x0 -= dx
    dx = f(x0)/ForwardDiff.derivative(f,x0) 
    steps += 1
  end 
  x0
end

newton (generic function with 1 method)

In [11]:
newton(x->x^2-5,1)

2.2360688956433634

Note: we haven't used the `Root` datatype developed in Chapter 12 for simplicity.  An exercise would be to combine this.

What if we want the root tolerance (1e-6) to be better or to run more steps than 10.  These are hard coded parameters that would have to be updated in the newton function.  A much better way to do this is to add these to the argument list, but with default values.  Julia calls these optional arguments.  Consider this rewrite:

In [12]:
function newton(f::Function, x0::Number, tol=1e-6, max_steps=10)
  local dx = f(x0)/ForwardDiff.derivative(f,x0)
  local steps = 0
  while abs(dx)>tol && steps<max_steps
    x0 -= dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  x0
end

newton (generic function with 3 methods)

In [13]:
methods(newton)

# 3 methods for generic function "newton":
[1] newton(f::Function, x0::Number) in Main at In[12]:1
[2] newton(f::Function, x0::Number, tol) in Main at In[12]:1
[3] newton(f::Function, x0::Number, tol, max_steps) in Main at In[12]:1

First, notice that there are 3 methods now. What are the signatures of newton for this?

Now, notice that we can change the tolerance with:

In [14]:
newton(x->x^2-5,1,1e-12)

2.236067977499978

But a problem is that to change the maximum number of steps, we need to change both:

In [15]:
newton(x->x^2-5,1,1e-12,20)

2.236067977499978

We don't need to set the argument types for optional arguments.  Julia will figure out the type by the number that you put in.  The example above, `tol` will be a Float and `max_steps` will be an Integer. 

We should also check these new arguments have proper types (each should be positive) with:

In [17]:
function newton(f::Function, x0::Number,tol=1e-6,max_steps=10)
  tol > 0 || throw(ArgumentError("The parameter tol must be positive"))
  max_steps > 0 || throw(ArgumentError("The parameter max_steps must be positive"))
  local dx = f(x0)/ForwardDiff.derivative(f,x0)
  local steps = 0
  while abs(dx)>tol && steps<max_steps
    x0 -= dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  x0
end

newton (generic function with 3 methods)

In [18]:
newton(x->x^2-5,1,-0.01)

LoadError: ArgumentError: The parameter tol must be positive

#### 16.3: Handling Special Cases

In [19]:
newton(x->x^2+2,1)

0.5353752385394379

In [20]:
using ForwardDiff
function newton(f::Function, x0::Number,tol=1e-6,max_steps=10)
  tol > 0 || throw(ArgumentError("The parameter tol must be positive"))
  max_steps > 0 || throw(ArgumentError("The parameter max_steps must be positive"))
  local dx = f(x0)/ForwardDiff.derivative(f,x0)
  local steps = 0
  while abs(dx)>tol && steps<max_steps
    x0 -= dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  local error = "The maximum number of steps: $max_steps was reached without convergence"
  steps < max_steps || throw(ErrorException(error))
  x0
end

newton (generic function with 3 methods)

In [21]:
newton(x->x^2+2,1)

LoadError: The maximum number of steps: 10 was reached without convergence

#### 16.4: Keyword Arguments

Instead of optional arguments (especially if there are more than one), it often better to use keyword arugments in which you can change any of the arguments in any order. To do this, we need to use the keyword when calling the function.  To change from optional arguments to keyword arguments, separate the regular arguments from the keyword ones with a semicolon (;)

In [22]:
function newton(f::Function, x0::Number; tol=1e-6, max_steps=10)
  tol > 0 || throw(ArgumentError("The parameter tol much be positive")) 
  max_steps > 0 || throw(ArgumentError("The parameter max_steps much be positive"))
  local x1 = x0
  local dx = f(x0)/ForwardDiff.derivative(f,x0) 
  local steps = 0
  while abs(dx)>tol && steps<max_steps
    x0 -= dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  local error = "The maximum number of steps: $max_steps was reached without convergence"
  steps < max_steps || throw(ErrorException(error))
  x0
end

newton (generic function with 3 methods)

In [23]:
newton(x->x^2-5,1,max_steps=5)

2.2360688956433634

#### 16.5: Parametric Methods

In Chapter 4, we saw multiple dispatch for julia.  Let's look at another example of this.  This will find the maximum value with different arguments put in. 

If we have two values we can do:

In [1]:
findMax(x::Real,y::Real) = x > y ? x : y

findMax (generic function with 1 method)

In [2]:
findMax(3,4)

4

As as we saw earlier, if we want any number of arguments, we could do:

In [3]:
function findMax(nums::Real...)
  local max = -Inf
  for num in nums
    if num > max
      max = num
    end
  end
  max
end

findMax (generic function with 2 methods)

In [4]:
findMax(1,6,2,3,-9,11//2,5.6)

6

It makes sense to also write a `findMax` function that takes an array of values:

In [5]:
function findMax(arr::Vector{Real})
  local max = -Inf
  for num in arr
    if num > max
      max = num
    end
  end
  max
end

findMax (generic function with 3 methods)

Let's say we want to find the max of `[-3,2,4,7,5]`.  It seems like the following should work:

In [6]:
findMax([-3,2,4,7,5])

LoadError: MethodError: no method matching findMax(::Vector{Int64})
[0mClosest candidates are:
[0m  findMax([91m::Real[39m, [91m::Real[39m) at In[1]:1
[0m  findMax([91m::Real...[39m) at In[3]:1
[0m  findMax([91m::Vector{Real}[39m) at In[5]:1

In [7]:
Int64 <: Real

true

In [8]:
Vector{Int64} <: Vector{Real}

false

However, notice that the error says that it can't find a `findMax` function with argument `Vector{Int64}`.  

We could create functions to handle arrays of integers, floats, and other types that we need, but julia has what is called parametric methods that handles all of these together:

In [9]:
function findMax(arr::Vector{T}) where {T <: Real}
  local max = -Inf
  for num in arr
    if num > max
      max = num
    end
  end
  max
end

findMax (generic function with 4 methods)

which can now handle any array with any subtype of `Real`.

In [10]:
findMax([-3,2,4,7,5])

7

In [11]:
typeof([-3,2,4,7,5])

Vector{Int64} (alias for Array{Int64, 1})

In [12]:
findMax([pi,1.0,-6.3, 6/8])

3.141592653589793

In [13]:
findMax([big(3),big(9)])

9

In [14]:
findMax([6//7, -99//8, 4//5])

6//7